In [1]:
#!curl -o duckdb.jar "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.0M  100 61.0M    0     0  12.8M      0  0:00:04  0:00:04 --:--:-- 12.8M


In [21]:
import duckdb
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
sc = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/13 23:56:43 WARN Utils: Your hostname, Ruths-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.28.41 instead (on interface en0)
24/04/13 23:56:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/13 23:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
paths = ['formatted_zone/compravenda_sup.db', 'formatted_zone/rent_price.db', 'formatted_zone/renda.db']
col_numeriques = {'formatted_zone/compravenda_sup.db': ['Nombre']
                  , 'formatted_zone/renda.db': ['Import_Euros']
                , 'formatted_zone/rent_price.db': ['Price']}

#### Mirar coses

In [79]:
for path in paths:

    #llegim db
    RDD=sc.read \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{path}") \
        .option("dbtable", "RDD") \
        .option("driver", "org.duckdb.DuckDBDriver").load()
    RDD.show()
    if "Superfície_mitjana_(m2_construïts)" in RDD.columns:
        valores_unicos = RDD.select('Superfície_mitjana_(m2_construïts)').distinct().collect()
        for valor in valores_unicos:
            print(valor)
    elif "Average _rent" in RDD.columns:
        valores_unicos = RDD.select('Average _rent').distinct().collect()
        for valor in valores_unicos:
            print(valor)
    
    #per guardar:
    '''RDD.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{path}") \
        .option("dbtable", "RDD") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .save()'''
            

+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
| Any|Trimestre|Codi_Districte| Nom_Districte|Codi_Barri|           Nom_Barri|Superfície_mitjana_(m2_construïts)|Nombre|
+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
|2023|        1|             1|  Ciutat Vella|         1|            el Raval|                             Total|  63.8|
|2023|        1|             1|  Ciutat Vella|         2|      el Barri Gòtic|                             Total|  82.9|
|2023|        1|             1|  Ciutat Vella|         3|      la Barceloneta|                             Total|  41.3|
|2023|        1|             1|  Ciutat Vella|         4|Sant Pere, Santa ...|                             Total|  70.9|
|2023|        1|             2|      Eixample|         5|       el Fort Pienc|                             Total|  80.6|
|2023|        1|             2| 

#### Prova codi esteban

In [71]:
for path in paths:
    RDD=sc.read \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{path}") \
        .option("dbtable", "RDD") \
        .option("driver", "org.duckdb.DuckDBDriver").load()
   
    for columna in col_numeriques[path]:
            valor_min = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
                .map(lambda x: (1, x[columna]))\
                .mapValues(lambda x: float(x) if type(x) == str else x)\
                .reduceByKey(lambda x, y: min(x,y)).collect()[0][1]
            valor_max = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
                .map(lambda x: (1, float(x[columna])))\
                .reduceByKey(lambda x, y: max(x,y)).collect()[0][1]
            
            RDD = RDD.withColumn(columna, (RDD[columna] - valor_min)/ (valor_max - valor_min))


/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")


TypeError: unsupported operand type(s) for -: 'float' and 'decimal.Decimal'

### Proves Denial constraints

In [76]:
path = paths[1]
RDD=sc.read \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{path}") \
        .option("dbtable", "RDD") \
        .option("driver", "org.duckdb.DuckDBDriver").load()


Aquí aniria codis de barris únics

In [63]:
# Denial Constraint: Unique Employee IDs
duplicate_ids = df.mapInArrow(lambda x: (x.select('Nom_Barri'), 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1)
if duplicate_ids.isEmpty():
    print("Data quality rule satisfied: Unique Employee IDs")
else:
    print("Data quality rule violated: Duplicate Employee IDs found")
    # Action to clean the data
    # For example, remove duplicates or keep only one occurrence


TypeError: mapInArrow() missing 1 required positional argument: 'schema'

La nateja del average rent

In [81]:
filtered_rdd = RDD.rdd.filter(lambda f: f["Average _rent"] == 'average rent (euro/month)')

# Replace values in the "Average _rent" column with "euro/month"
transformed_rdd = filtered_rdd.map(lambda x: {**x, "Average _rent": "euro/month"})

# Show the resulting RDD
transformed_rdd.collect()


24/04/14 01:37:46 ERROR Executor: Exception in task 0.0 in stage 65.0 (TID 57)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2364, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'Average _rent' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = li

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 65.0 failed 1 times, most recent failure: Lost task 0.0 in stage 65.0 (TID 57) (192.168.28.41 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2364, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'Average _rent' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 123